# 297. Serialize and Deserialize Binary Tree


## Topic Alignment
- Serialization is required when persisting model topologies or transmitting tree structures between services.
- Reliable encode/decode routines underpin configuration snapshots and replayable experiments.


## Metadata Summary
- Source: [Serialize and Deserialize Binary Tree](https://leetcode.com/problems/serialize-and-deserialize-binary-tree/)
- Tags: `Binary Tree`, `Design`, `BFS`
- Difficulty: Hard
- Recommended Priority: High


## Problem Statement
Design an algorithm to serialize and deserialize a binary tree. Serialization converts the tree to a string, and deserialization reconstructs the exact original tree from that string.


## Progressive Hints
- Hint 1: Pick a traversal order (preorder is common) and include null markers to preserve structure.
- Hint 2: During deserialization, consume the serialized tokens in the same order they were produced.
- Hint 3: Use iterators to avoid repeated slicing or popping from the front of a list.


## Solution Overview
Serialize the tree via preorder traversal, appending a sentinel such as `#` for null children. Join the tokens with a delimiter. To deserialize, split the string back into tokens and rebuild the tree recursively, consuming tokens in preorder.


## Detailed Explanation
1. For serialization, define a helper that appends the node value to a list and recurses on left then right.
2. When encountering a `None`, append the sentinel instead of recursing further.
3. After traversal, join the token list with commas to produce the serialized string.
4. For deserialization, convert the string back into an iterator of tokens.
5. Recursively consume tokens: if the current token is the sentinel, return `None`; otherwise create a node and build its left and right children.
6. Return the root reconstructed from the iterator.


## Complexity Trade-off Table
| Approach | Time Complexity | Space Complexity | Notes |
| --- | --- | --- | --- |
| Preorder with sentinel | O(n) | O(n) | Simple and bijective representation |
| Level-order serialization | O(n) | O(n) | Produces a breadth-first form; longer strings for sparse trees |
| Binary encoding (custom) | O(n) | O(n) | Compact but harder to debug |


## Reference Implementation


In [ ]:
from collections import deque
from typing import Optional


class TreeNode:
    def __init__(self, val: int = 0, left: Optional['TreeNode'] = None, right: Optional['TreeNode'] = None):
        self.val = val
        self.left = left
        self.right = right


class Codec:
    def serialize(self, root: Optional[TreeNode]) -> str:
        if not root:
            return ''
        queue: deque[Optional[TreeNode]] = deque([root])
        output: list[str] = []
        while queue:
            node = queue.popleft()
            if node:
                output.append(str(node.val))
                queue.append(node.left)
                queue.append(node.right)
            else:
                output.append('#')
        while output and output[-1] == '#':
            output.pop()
        return ','.join(output)

    def deserialize(self, data: str) -> Optional[TreeNode]:
        if not data:
            return None
        values = data.split(',')
        root = TreeNode(int(values[0]))
        queue: deque[TreeNode] = deque([root])
        index = 1
        while queue and index < len(values):
            node = queue.popleft()
            if index < len(values) and values[index] != '#':
                node.left = TreeNode(int(values[index]))
                queue.append(node.left)
            index += 1
            if index < len(values) and values[index] != '#':
                node.right = TreeNode(int(values[index]))
                queue.append(node.right)
            index += 1
        return root


## Validation


In [ ]:
def compare(a: Optional[TreeNode], b: Optional[TreeNode]) -> bool:
    if not a and not b:
        return True
    if not a or not b:
        return False
    return a.val == b.val and compare(a.left, b.left) and compare(a.right, b.right)

codec = Codec()
root = TreeNode(1, TreeNode(2), TreeNode(3, TreeNode(4), TreeNode(5)))
data = codec.serialize(root)
restored = codec.deserialize(data)
assert compare(root, restored), 'Tree should be identical after round trip'
assert codec.serialize(None) == '', 'Empty tree serialization should be empty string'


## Complexity Analysis
- Time: O(n) to serialize and O(n) to deserialize, visiting every node once.
- Space: O(n) for the token list or recursion stack during deserialization.
- Bottleneck: String size grows linearly with the number of nodes and sentinels.


## Edge Cases & Pitfalls
- Ensure leaf children emit sentinel markers; otherwise structure cannot be recovered.
- Avoid using characters that conflict with the chosen delimiter.
- Handle empty input by returning `None` immediately.


## Follow-up Variants
- Serialize into binary format for compact storage when bandwidth is constrained.
- Support streaming by writing tokens to a generator rather than building a full list.
- Add version headers or checksum fields to detect corruption during transport.


## Takeaways
- Matching traversal order between serialization and deserialization guarantees correctness.
- Sentinels are a simple yet powerful way to preserve null structure.
- Iterator-based reconstruction avoids quadratic behavior from repeated list pops.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| 449 | Serialize and Deserialize BST | Exploits BST property to omit sentinels |
| 652 | Find Duplicate Subtrees | Serialization to detect duplicates |
| 428 | Serialize and Deserialize N-ary Tree | Extends preorder to variable arity |
